In [1]:
from pyspark.sql import SparkSession

# Create spark session
spark = (
    SparkSession.builder.appName("MAST30034_Yellow_Taxi_Preprocessing_Feature_Engineering")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/08/22 16:23:25 WARN Utils: Your hostname, LAPTOP-VAB0S7AL resolves to a loopback address: 127.0.1.1; using 172.27.239.27 instead (on interface eth0)
22/08/22 16:23:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/08/22 16:23:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/22 16:23:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Read in 2019 Taxi data -> this will be used for training
sdf_full = spark.read.parquet('../data/raw/tlc_data/tlc_data_yellow/2019*')

-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2019-03-01 00:24:41 
 tpep_dropoff_datetime | 2019-03-01 00:25:31 
 passenger_count       | 1.0                 
 trip_distance         | 0.0                 
 RatecodeID            | 1.0                 
 store_and_fwd_flag    | N                   
 PULocationID          | 145                 
 DOLocationID          | 145                 
 payment_type          | 2                   
 fare_amount           | 2.5                 
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 3.8                 
 congestion_surcharge  | 0.0                 
 airport_fee           | null                
only showing top 1 row



In [ ]:
sdf_full.count()

In [ ]:
sdf_full.printSchema()

In [3]:
from pyspark.sql import functions as F
from scripts._global_vars import *
from scripts.collate import drop_cast_and_create_taxi

In [5]:
sdf = drop_cast_and_create_taxi(sdf_full)

In [6]:
# verify
#sdf.printSchema()
#sdf.limit(5)

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- PU_datetime: date (nullable = true)
 |-- DO_datetime: date (nullable = true)
 |-- PU_hourofday: integer (nullable = true)
 |-- DO_hourofday: integer (nullable = true)
 |-- PU_dayofweek: integer (nullable = true)
 |-- DO_dayofweek: integer (nullable = true)
 |-- PU_dayofmonth: integer (nullable = true)
 |-- DO_dayofmonth: integer (nullable = true)
 |-- PU_month: integer (nullable = true)
 |-- DO_month: integer (nullable = true)


22/08/22 16:24:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,tip_amount,congestion_surcharge,PU_datetime,DO_datetime,PU_hourofday,DO_hourofday,PU_dayofweek,DO_dayofweek,PU_dayofmonth,DO_dayofmonth,PU_month,DO_month,trip_time_minutes,trip_speed_mph,fare_per_minute,hour_of_day_of_year
1,2019-03-01 00:24:41,2019-03-01 00:25:31,1.0,0.0,1,145,145,2,2.5,0.0,0.0,2019-03-01,2019-03-01,0,0,6,1,1,1,3,3,0.8333333333333334,0.0,3.0,2019-03-01 00:00:00
1,2019-03-01 00:25:27,2019-03-01 00:36:37,2.0,3.7,1,95,130,1,13.0,0.7,0.0,2019-03-01,2019-03-01,0,0,6,1,1,1,3,3,11.166666666666666,19.880597014925378,1.164179104477612,2019-03-01 00:00:00
1,2019-03-01 00:05:21,2019-03-01 00:38:23,1.0,14.1,1,249,28,1,41.0,10.1,2.5,2019-03-01,2019-03-01,0,0,6,1,1,1,3,3,33.03333333333333,25.610494450050453,1.24117053481332,2019-03-01 00:00:00
1,2019-03-01 00:48:55,2019-03-01 01:06:03,1.0,9.6,1,138,98,2,27.0,0.0,0.0,2019-03-01,2019-03-01,0,1,6,1,1,1,3,3,17.133333333333333,33.61867704280156,1.575875486381323,2019-03-01 00:00:00
1,2019-03-01 00:11:42,2019-03-01 00:16:40,1.0,0.8,1,48,48,1,5.5,3.0,2.5,2019-03-01,2019-03-01,0,0,6,1,1,1,3,3,4.966666666666667,9.664429530201343,1.1073825503355705,2019-03-01 00:00:00


In [7]:
# Pre cleaning descriptive statistics
sdf.describe(*non_categorical_features).show()

+-------+------------------+------------------+------------------+------------------+--------------------+
|summary|       fare_amount|     trip_distance|   passenger_count|        tip_amount|congestion_surcharge|
+-------+------------------+------------------+------------------+------------------+--------------------+
|  count|          84598444|          84598444|          84154061|          84598444|            79297843|
|   mean|13.412639732835764|3.0183506184817515|1.5626654190817957| 2.190078737505638|  2.1949917301029234|
| stddev|174.17668755385404| 8.093902044464816|1.2079081585219809|15.638996154306168|  0.8296498809008713|
|    min|           -1856.0|         -37264.53|               0.0|            -221.0|                -2.5|
|    max|          943274.8|          45977.22|               9.0|         141492.02|                 4.5|
+-------+------------------+------------------+------------------+------------------+--------------------+



### Weather data aggregation + joining

In [8]:
# read in weather data
weather_date_path = "../data/raw/nyc_weather_date/NYC.csv"
weather_sdf = spark.read.csv(weather_date_path, header=True, inferSchema=True)

In [10]:
weather_sdf.printSchema()

root
 |-- station: string (nullable = true)
 |-- valid: string (nullable = true)
 |-- lon: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- tmpf: string (nullable = true)
 |-- dwpf: string (nullable = true)
 |-- relh: string (nullable = true)
 |-- drct: string (nullable = true)
 |-- sknt: string (nullable = true)
 |-- p01i: string (nullable = true)
 |-- alti: string (nullable = true)
 |-- mslp: string (nullable = true)
 |-- vsby: string (nullable = true)
 |-- gust: string (nullable = true)
 |-- skyc1: string (nullable = true)
 |-- skyc2: string (nullable = true)
 |-- skyc3: string (nullable = true)
 |-- skyc4: string (nullable = true)
 |-- skyl1: string (nullable = true)
 |-- skyl2: string (nullable = true)
 |-- skyl3: string (nullable = true)
 |-- skyl4: string (nullable = true)
 |-- wxcodes: string (nullable = true)
 |-- ice_accretion_1hr: string (nullable = true)
 |-- ice_accretion_3hr: string (nullable = true)
 |-- ice_accretion_6hr: string (nullable = true)
 |-- pe

In [14]:
from scripts.collate import weather_process

In [15]:
weather_sdf = weather_process(weather_sdf)

In [16]:
weather_sdf.count()
weather_sdf.printSchema()
# few missing values -> we dont have to worry -> time to join datasets

root
 |-- hour_of_day_of_year: timestamp (nullable = true)
 |-- tmpf: double (nullable = true)
 |-- dwpf: double (nullable = true)
 |-- relh: double (nullable = true)



In [18]:
export_relative_dir = '../data/curated/'

In [19]:
# Export processed weather data
weather_sdf.write.mode('overwrite').csv(export_relative_dir + "weather_data_clean.csv")

In [20]:
# Collate weather and NYCTLC dataset
combined_sdf = sdf.join(weather_sdf, on=['hour_of_day_of_year'], how='inner')

In [23]:
combined_sdf.write.mode('overwrite').parquet(export_relative_dir + "yt2019_feature_eng.parquet")
print("Export completed")

export completed


In [24]:
spark.sql("CLEAR CACHE")
spark.stop()